In [68]:
# Import the modules
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [69]:
# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///../.sqlite")

In [70]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(
    Path('..\project 4\Clean_datasets\mergefinal.csv')   
)

# Review the DataFrame
df.head()


,LOCATION,Year,G20 Value,Education Value,GDP Value,LifeExp Value,Population Value,TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Unemployment Value
0,Australia,1967,11.799078,NaN,3447.128012,70.8,11.799078,2.85,18.0,1.875000
1,Australia,1976,14.033083,NaN,7396.879985,72.8,14.033083,2.06,13.6,4.750000
2,Australia,1981,14.923260,NaN,11956.491969,74.8,14.923260,1.94,10.4,5.783571
3,Australia,1982,15.184247,NaN,11988.843953,74.6,15.184247,1.94,10.1,7.156132
4,Australia,1983,15.393472,NaN,12781.125663,75.4,15.393472,1.93,9.7,9.961594


In [71]:
# Convert categorical data to numeric with `pd.get_dummies`
df.info()
dummies_df = pd.get_dummies(df,columns = ['LOCATION'])
dummies_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 10 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   LOCATION                                            419 non-null    object 
 1   Year                                                419 non-null    int64  
 2   G20 Value                                           419 non-null    float64
 3   Education Value                                     307 non-null    float64
 4   GDP Value                                           419 non-null    float64
 5   LifeExp Value                                       419 non-null    float64
 6   Population Value                                    419 non-null    float64
 7   TotalFertilityRate(LiveBirthsPerWoman)              419 non-null    float64
 8   InfantMortalityRate(InfantDeathsPer1000LiveBirths)  419 non-null    float64
 9  

,Year,G20 Value,Education Value,GDP Value,LifeExp Value,Population Value,TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Unemployment Value,LOCATION_Australia,LOCATION_Canada,LOCATION_France,LOCATION_Germany,LOCATION_Italy,LOCATION_Japan,LOCATION_Mexico,LOCATION_Republic of Korea,LOCATION_Türkiye,LOCATION_United Kingdom,LOCATION_United States of America
0,1967,11.799078,NaN,3447.128012,70.8,11.799078,2.85,18.0,1.875000,1,0,0,0,0,0,0,0,0,0,0
1,1976,14.033083,NaN,7396.879985,72.8,14.033083,2.06,13.6,4.750000,1,0,0,0,0,0,0,0,0,0,0
2,1981,14.923260,NaN,11956.491969,74.8,14.923260,1.94,10.4,5.783571,1,0,0,0,0,0,0,0,0,0,0
3,1982,15.184247,NaN,11988.843953,74.6,15.184247,1.94,10.1,7.156132,1,0,0,0,0,0,0,0,0,0,0
4,1983,15.393472,NaN,12781.125663,75.4,15.393472,1.93,9.7,9.961594,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,2017,325.122128,47.757000,59589.233923,78.6,325.122128,1.76,5.7,4.350000,0,0,0,0,0,0,0,0,0,0,1
415,2018,326.838199,49.373737,62449.612827,78.7,326.838199,1.72,5.6,3.900000,0,0,0,0,0,0,0,0,0,0,1
416,2019,328.329953,50.380188,64690.272395,78.8,328.329953,1.69,5.5,3.666667,0,0,0,0,0,0,0,0,0,0,1
417,2020,331.511512,51.856071,63480.861222,77.0,331.511512,1.64,5.5,8.091666,0,0,0,0,0,0,0,0,0,0,1


In [72]:
dummies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 20 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Year                                                419 non-null    int64  
 1   G20 Value                                           419 non-null    float64
 2   Education Value                                     307 non-null    float64
 3   GDP Value                                           419 non-null    float64
 4   LifeExp Value                                       419 non-null    float64
 5   Population Value                                    419 non-null    float64
 6   TotalFertilityRate(LiveBirthsPerWoman)              419 non-null    float64
 7   InfantMortalityRate(InfantDeathsPer1000LiveBirths)  419 non-null    float64
 8   Unemployment Value                                  419 non-null    float64
 9  

In [73]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['TotalFertilityRate(LiveBirthsPerWoman)']
X = dummies_df.drop(columns=['TotalFertilityRate(LiveBirthsPerWoman)', 'Education Value'])
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [74]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [75]:
# Determine the number of unique values in each column.
dummies_df.nunique()


Year                                                   53
G20 Value                                             419
Education Value                                       307
GDP Value                                             419
LifeExp Value                                         130
Population Value                                      419
TotalFertilityRate(LiveBirthsPerWoman)                140
InfantMortalityRate(InfantDeathsPer1000LiveBirths)    148
Unemployment Value                                    375
LOCATION_Australia                                      2
LOCATION_Canada                                         2
LOCATION_France                                         2
LOCATION_Germany                                        2
LOCATION_Italy                                          2
LOCATION_Japan                                          2
LOCATION_Mexico                                         2
LOCATION_Republic of Korea                              2
LOCATION_Türki

In [76]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
# Create a regression model
regr = RandomForestRegressor(max_depth=2, random_state=0)
# Train the model
regr.fit(X_train_scaled, y_train)


RandomForestRegressor(max_depth=2, random_state=0)

In [79]:
y_pred = regr.predict(X_test_scaled)
y_pred


array([2.84929477, 1.6142031 , 1.69640777, 1.78890323, 3.20101936,
       2.36958445, 1.78558136, 1.75941797, 3.13973314, 1.72492796,
       1.69472114, 3.18557797, 1.53083415, 1.64270467, 1.73658605,
       1.50062732, 1.78558136, 1.75941797, 2.39222636, 1.64270467,
       1.50062732, 1.6142031 , 1.62598514, 1.5541825 , 1.53083415,
       1.6142031 , 1.59436078, 1.79651597, 1.55171122, 1.78558136,
       2.44432743, 1.68474968, 1.78558136, 2.35403197, 1.6142031 ,
       1.64270467, 1.78558136, 1.6142031 , 1.62598514, 1.78558136,
       1.6142031 , 1.6142031 , 1.64270467, 1.6142031 , 1.6142031 ,
       1.78890323, 1.6142031 , 1.6142031 , 1.78558136, 2.44973224,
       1.6142031 , 1.52150439, 1.78558136, 1.62598514, 2.37883406,
       1.72492796, 1.56383435, 1.78558136, 3.20101936, 2.49440508,
       1.49065587, 1.61111801, 1.6142031 , 1.64270467, 1.6142031 ,
       1.78558136, 1.78558136, 1.64270467, 1.6142031 , 1.78558136,
       1.49065587, 1.53083415, 1.79771098, 1.6142031 , 1.78558

In [80]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
score = regr.score(X_test_scaled, y_test, sample_weight=None)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
std = np.std(y)
# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.7360764251668273.
The r2 is 0.7360764251668273.
The mean squared error is 0.060816528162813396.
The root mean squared error is 0.24661007311708374.
The standard deviation is 0.41530337320156274.
